In [1]:
import pandas as pd
import numpy as np
data=pd.read_csv('ratings_train.csv')
len(data['rating'].isnull())

89992

In [2]:
print(len(data["item_id"].unique()))
print(max(data['item_id']))

1662
1680


len(data["item_id"].unique() gives 1662 and max(data['item_id']) gives 1680 meaning there are users who havent reviewed any movies and we have to predict ratings for them so adjacency matrix of shape (943,1681)

In [3]:
from scipy.sparse import csr_matrix
n=(data["user_id"].unique().shape[0])
m=max(data['item_id'])+1
adjacency_matrix = np.zeros((n,m))
for row in data.itertuples(index = False):
    adjacency_matrix[row.user_id - 1, row.item_id - 1] = row.rating

In [4]:
print(len(data[data['item_id'].isnull()]))
print(len(data[data['user_id'].isnull()]))
#no null values in the dataset

0
0


In [5]:
def grader_matrix(matrix):
    assert(matrix.shape==(943,1681))
    return True
grader_matrix(adjacency_matrix)

True

In [6]:
from sklearn.utils.extmath import randomized_svd
import numpy as np 
U, Sigma, VT = randomized_svd(adjacency_matrix, n_components=100,n_iter=5, random_state=None)
print(U.shape)
print(Sigma.shape)
print(VT.T.shape)

(943, 100)
(100,)
(1681, 100)


In [7]:
def m_u(ratings):
    return np.mean(ratings)
mu=m_u(data['rating'])
print("mean_rating",mu)

mean_rating 3.529480398257623


In [8]:
def grader_mean(mu):
    assert(np.round(mu,3)==3.529)
    return True
mu=m_u(data['rating'])
grader_mean(mu)

True

In [9]:
def initialize(dim):
    '''In this function, we will initialize bias value 'B' and 'C'.'''
    # initalize the value to zeros 
    # return output as a list of zeros 
    return np.zeros_like(dim)

In [10]:
dim= adjacency_matrix[:,0]# give the number of dimensions for b_i (Here b_i corresponds to users)
b_i=initialize(dim)#initialize bias vector for each user
len(b_i)

943

In [11]:
dim= adjacency_matrix[0]# give the number of dimensions for c_j (Here c_j corresponds to movies)
c_j=initialize(dim)#bias vector coresponding to movies
len(c_j)

1681

In [12]:
def grader_dim(b_i,c_j):
    assert(len(b_i)==943 and np.sum(b_i)==0)
    assert(len(c_j)==1681 and np.sum(c_j)==0)
    return True
grader_dim(b_i,c_j)

True

In [13]:
def derivative_db(user_id,item_id,rating,U,V,mu,alpha):
    '''In this function, we will compute dL/db_i'''
    der_b=0
    b=initialize(U[:,0])
    c=initialize(V.T[:,0])
    der_b=2*alpha*b[user_id-1] -2*(rating-mu-b[user_id-1]-c[item_id-1]-np.dot(U[user_id-1].T,V[:,item_id-1]))     
    return der_b


In [14]:
def grader_db(value):
    assert(np.round(value,3)==-0.931)
    return True
U1, Sigma, V1 = randomized_svd(adjacency_matrix, n_components=2,n_iter=5, random_state=24)
# Please don't change random state
# Here we are considering n_componets = 2 for our convinence
alpha=0.01 
value=derivative_db(312,98,4,U1,V1,mu,alpha)
grader_db(value)

True

In [15]:
def derivative_dc(user_id,item_id,rating,U,V,mu):
    '''In this function, we will compute dL/dc_j'''
    der_c=0
    b=initialize(U[:,0])
    c=initialize(V.T[:,0])
    der_c=2*0.01*c[item_id-1] -2*(rating-mu-b[user_id-1]-c[item_id-1]-np.dot(U[user_id-1].T,V[:,item_id-1]))     
    return der_c



In [16]:
def grader_dc(value):
    assert(np.round(value,3)==-2.929)
    return True
U1, Sigma, V1 = randomized_svd(adjacency_matrix, n_components=2,n_iter=5, random_state=24)
# Please don't change random state
# Here we are considering n_componets = 2 for our convinence
r=0.01 
value=derivative_dc(58,504,5,U1,V1,mu)
grader_dc(value)

True

In [17]:
from tqdm import tqdm
n_epoch=5
b=initialize(U[:,0])
c=initialize(VT.T[:,0])
user_id=data["user_id"].tolist()
item_id=data["item_id"].tolist()
rate=data["rating"].tolist()
for each in tqdm(range(n_epoch)):
    for i in range(len(data)):
        b[data["user_id"][i]-1]+= -0.001*derivative_db(data["user_id"][i],data["item_id"][i],data["rating"][i],U,VT,mu,alpha)

        c[data["item_id"][i]-1]+= -0.001*derivative_dc(data["user_id"][i],data["item_id"][i],data["rating"][i],U,VT,mu)


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:11<00:00, 38.22s/it]


In [18]:
mse=0
predicted=[]
for i in range(len(data)):
    y_predicted=mu+b[data["user_id"][i]-1] + c[data["item_id"][i]-1]+ np.dot(U[data["user_id"][i]-1].T,VT[:,data["item_id"][i]-1])
    mse+=(y_predicted-data["rating"][i])**2 #mse of predicted ratings
    predicted.append(y_predicted)

In [19]:
print("mse of predicted ratings ",mse/len(data))

mse of predicted ratings  2.295108722830065


In [20]:
print("predicted rating")
predicted[:5]

predicted rating


[3.0788533517790326,
 4.451341194948581,
 3.556957111826103,
 3.902759622900678,
 5.2110771106252125]

In [21]:
data["rating"][:5]

0    3
1    5
2    4
3    4
4    5
Name: rating, dtype: int64

In [22]:
#PROBLEM 2 PREDICTING SEX OF USER
data_2=pd.read_csv('user_info.csv.txt')


In [23]:
data_2=data_2.drop(['orig_user_id'], axis=1)
data_2=data_2.drop(['age'], axis=1)
data_2[:5]

,user_id,is_male
0,0,1
1,1,0
2,2,1
3,3,1
4,4,0


In [33]:
import sklearn
from sklearn.preprocessing import StandardScaler
data_U=sklearn.preprocessing.normalize(U)
data_VT=sklearn.preprocessing.normalize(VT)
data=data.merge(data_2, left_on='user_id', right_on='user_id')#merging both the datasets
data.head()

,user_id,item_id,rating,is_male_x,is_male_y
0,772,36,3,1,1
1,772,175,4,1,1
2,772,1168,3,1,1
3,772,208,5,1,1
4,772,322,3,1,1


In [25]:
len(data["is_male"])

89992

In [26]:
n_epoch=5
mu_2=m_u(data['is_male'])
b=initialize(data_U[:,0])
c=initialize(data_VT.T[:,0])
for each in tqdm(range(n_epoch)):
    for i in range(len(data)):
        b[data["user_id"][i]-1]+= -0.0002*derivative_db(data["user_id"][i],data["item_id"][i],data["is_male"][i],data_U,data_VT,mu_2,alpha)

        c[data["item_id"][i]-1]+= -0.0002*derivative_dc(data["user_id"][i],data["item_id"][i],data["is_male"][i],data_U,data_VT,mu_2)


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [03:01<00:00, 36.28s/it]


In [27]:
y_predicted=0
mse=0
predicted_2=[]
for i in range(len(data)):
    y_predicted=mu_2+b[data["user_id"][i]-1] + c[data["item_id"][i]-1]+ np.dot(data_U[data["user_id"][i]-1].T,data_VT[:,data["item_id"][i]-1])
    mse+=(y_predicted-data["is_male"][i])**2
    predicted_2.append(y_predicted)
    

In [28]:
mse_2=0
predicted=[]
for i in range(len(data)):
    y_predicted=mu_2+b[data["user_id"][i]-1] + c[data["item_id"][i]-1]+ np.dot(U[data["user_id"][i]-1].T,VT[:,data["item_id"][i]-1])
    mse_2+=(y_predicted-data["is_male"][i])**2
    predicted.append(y_predicted)

In [29]:
mse/len(data)

0.09684996612713985

In [30]:
from sklearn import metrics
y_2=[]
for i in range(len(predicted)):
    if predicted[i]<0.5:
        y_2.append(0)
    else:
        y_2.append(1)
print("accuracy Score for predicting males :{0:.4f}".format (metrics.accuracy_score(data["is_male"],y_2)*100))       

accuracy Score for predicting males :86.2499


In [31]:
len(data[data["is_male"]==0])

23123